# Grab A.I. for SEA Challenge: Traffic Management

Jayson Yodico <br>
Asian Institute of Management

## Problem Statement

Understanding congestion dynamics is indispensable in solving traffic congestion.

This project seeks to accurately forecast travel demand on a specific area and time based on historical Grab bookings.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pygeohash as gh

import scipy
import tensorflow.keras
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error

import itertools
import os

## PREPROCESSING

Upon inspection, it was observed that the are no demand data for some location-time bucket pairs. It is necessary to fill up missing values to make all geohash time series complete.

Since it was assumed that all demand data were included in the the timeframe of this training dataset, geohash-time bucket pairs with no demand data are assumed to be zero. This simply means that there is zero demand in that location and time bucket.

The purpose of the preprocessing is to fill up gaps in the time series and to make sure that values are arranged chronologically for each location. In addition, decoding of the geohashes  to coordinates (latitude, longitude) and included as additional fields in the processed dataset. The steps are outlined as follows:

*Note: The training dataset,* `train.csv`*, must be placed in the folder* `Traffic Management` *of the repository. The dataset can be downloaded from:* https://s3-ap-southeast-1.amazonaws.com/grab-aiforsea-dataset/traffic-management.zip.

In [13]:
# raw_df = pd.read_csv('Traffic Management/training.csv')
# raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'], format= '%H:%M').dt.time

1. Create dummy datetime values to serve as reference in ordering the `day` and `timestamp` columns.

In [14]:
# dates = pd.DataFrame()
# dates['dummy_date'] = pd.date_range(start=pd.datetime(2019, 1, 1),
#                               periods=len(raw_df.day.unique())+1)
# dates['day'] = np.arange(1, dates.shape[0] + 1)

2. With the dummy datetime values, the sequence can be ordered. For each location, `T_n` serves as an ID indicating the order a value appears in that location-time series.

In [5]:
"""
Create a reference table of chronological ordering indices (T_n) mapped to
timestamp and day values.
"""

timenum = pd.DataFrame(pd.date_range(start=dates.dummy_date.min(),
                                     end=dates.dummy_date.max(),
                                     freq='15min'), columns=['dummy_datetime'])

timenum['dummy_date'] = pd.to_datetime(timenum['dummy_datetime'].dt.date)
timenum['timestamp'] = timenum['dummy_datetime'].dt.time

timenum['T_n'] = np.arange(timenum.shape[0])
timenum = timenum.merge(dates, on='dummy_date', how='left')

del timenum['dummy_datetime'], timenum['dummy_date']
timenum.head()

,timestamp,T_n,day
0,00:00:00,0,1
1,00:15:00,1,1
2,00:30:00,2,1
3,00:45:00,3,1
4,01:00:00,4,1


In [7]:
"""
Merging the table with the original dataset
"""

df = raw_df.merge(timenum, on=['day', 'timestamp'], how='left')
df.head()

,geohash6,day,timestamp,demand,T_n
0,qp03wc,18,20:00:00,0.020072,1712
1,qp03pn,10,14:30:00,0.024721,922
2,qp09sw,9,06:15:00,0.102821,793
3,qp0991,32,05:00:00,0.088755,2996
4,qp090q,15,04:00:00,0.074468,1360


3. Filling in gaps in the time series with zeroes (i.e. no demand at that time and location) and decoding the geohashes into latitude and longitude.

In [8]:
g = df.groupby(['geohash6'])

all_data = []

for loc in g.groups.keys():
    
    test = g.get_group(loc)

    dummy = timenum[(timenum.T_n >= test.T_n.min())
                    & (timenum.T_n <= test.T_n.max())]
    
    # Fill gaps with zeroes
    dummy = dummy.merge(test[['T_n', 'demand']], on='T_n', how='left').fillna(0)
    dummy['geohash6'] = loc
    dummy['lat'] = gh.decode(loc)[0]
    dummy['long'] = gh.decode(loc)[1]
    
    all_data.append(dummy)

In [9]:
data = pd.concat(all_data)
data.head()

,timestamp,T_n,day,demand,geohash6,lat,long
0,02:45:00,11,1,0.020592,qp02yc,-5.48,90.7
1,03:00:00,12,1,0.010292,qp02yc,-5.48,90.7
2,03:15:00,13,1,0.000000,qp02yc,-5.48,90.7
3,03:30:00,14,1,0.000000,qp02yc,-5.48,90.7
4,03:45:00,15,1,0.000000,qp02yc,-5.48,90.7


4. Saving the processed file. This will be used to create models in the next section.

In [10]:
# data.to_csv('Traffic Management/training_processed.csv', index=False)

This process ensures that timestamps for every geohash is put together to create a geohash-time series.

## Feature Engineering and Model Development

The model development process is centered around three processes:
- **Denoising.** Applying a certain level of denoising using Fourier transform
- **Windowing**. Given historical values ($T_{n},T_{n-1},.., T_{n-k}$), predict future values ($T_{n+1},T_{n+2}.., T_{n+5}$)
- **Genetic Algorithm**. There are many possible model configurations if the data is to be feature-engineered in terms of denoising and windowing. The conventional approach is to use grid search or randomized search in finding the best denoising-windowing configuration. To make the search process more efficient and systematic, this project proposes to use genetic algorithm to find the best denoising-windowing configuration.
- **Model Fitting and Performance.** Each window-percentile combination is fitted on a 3-layer neural network model. 


Model is fitted on a training dataset and updated using a 

The details of the techniques used in this study are as follows:

### Fourier Transform

The first step is to denoise each geohash time series using Fourier transform. Used in signal processing, it postulates that any waveform whose value varies with time can be decomposed into simple waveforms of varying frequencies. From the time domain, the series is projected into the frequency domain though a spectrogram. Take for example a geohash time series:

<img src="sample_series.png" width="1500" />

in the time domain projected to the frequency domain. The image below represents the distribution of intensities of the frequency components that make up the time series.

<img src="projected series.png" width="700" />

Fourier transform works as a denoising method by silencing the components with low intensities. From this a threshold could in terms of a percentile value $q$, such that intensities lower than $q$ is set to zero. Higher levels of $q$ means a higher level of denoising. For example, when intensities below the 90th percentile are silenced, the resulting spectrogram is below:

<img src="projected series silenced.png" width="700" />

This silenced spectrogram, when brought back to the time domain, becomes the following series.

<img src="cleaned_series.png" width="1500" />

Notice how the denoised series is approximately the shape of the original series, but is less fluctuating. However, too much denoising could ignore the valuable fluctuations of a time series, such that when fitted to a machine learning model, could make less powerful predictions.

In this sense, a parameter to be tuned in this process is a *percentile value* denoting level of denoising.


In [ ]:
def denoise(series, pctile):

    """
    Denoises a time series by projecting the series to the frequency domain and
    silencing frequencies less than the frequency threshold.

    Parameters:

    series: ndarray
        - series to denoised

    pctile: int
        - percentile threshold of denoising. In the frequency domain, all
          frequency components with intensities less than th 
    """

    sff = scipy.fft(series)    
    abs_sff = abs(sff)
    sff[abs_sff < np.percentile(abs_sff, q=pctile)] = 0
    cleaned_series = np.abs(scipy.ifft(sff))

    return cleaned_series

In [44]:
def create_trainval_test(data, pctile, max_window, frac=0.001):

    i = 0
    g = data.groupby('geohash6')
    
    for each_gh in g.groups.keys():
        dummy = g.get_group(each_gh).copy()
        dummy['demand_fft'] = denoise(dummy.demand.values, pctile)

        for fwd in range(-2,-6,-1):
            dummy[f'demand+{-1*fwd}'] = dummy['demand'].shift(fwd+1)
            dummy[f'demand_fft+{-1*fwd}'] = dummy['demand_fft'].shift(fwd+1)

        for bwd in range(1, 6):
            dummy[f'demand-{bwd}'] = dummy['demand'].shift(bwd)

        for bwd in range(1, max_window + 1):
            dummy[f'demand_fft-{bwd}'] = dummy['demand_fft'].shift(bwd)

        dummy = dummy.dropna()
        len_dummy = dummy.shape[0]
        sp1, sp2 = int(len_dummy*0.6), int(len_dummy*0.8)

        trn = dummy.iloc[:sp1,:].sample(frac=frac)
        vl = dummy.iloc[sp1:sp2,:].sample(frac=frac)
        tst = dummy.iloc[sp2:,:].sample(frac=frac)

        if i == 0:
            trn.to_csv('train1.csv', index=False)
            vl.to_csv('val1.csv', index=False)
            tst.to_csv('test1.csv', index=False)

        else:
            trn.to_csv('train1.csv', mode='a', header=False, index=False)
            vl.to_csv('val1.csv', mode='a', header=False, index=False)
            tst.to_csv('test1.csv', mode='a', header=False, index=False)  
        
        i+=1
            
    train = pd.read_csv('train1.csv')
    val = pd.read_csv('val1.csv')
    test = pd.read_csv('test1.csv')
    
    return train, val, test

def split(train, val, test):

    toremove_feat = ['demand','demand+2', 'demand+3','demand+4', 'demand+5',
                     'demand_fft','demand_fft+2', 'demand_fft+3','demand_fft+4',
                     'demand_fft+5','demand-1', 'demand-2', 'demand-3',
                     'demand-4','demand-5', 'geohash6']

    targets = ['demand_fft', 'demand_fft+2', 'demand_fft+3', 'demand_fft+4',
               'demand_fft+5']
    
    tags = ['geohash6']

    targets_actual = ['demand', 'demand+2', 'demand+3', 'demand+4', 'demand+5']

    X_train = train.drop(toremove_feat, axis=1)
    y_train = train[targets]
    train_tag = train[tags]

    X_val = val.drop(toremove_feat, axis=1)
    y_val = val[targets]
    val_tag = val[tags]

    X_test = test.drop(toremove_feat, axis=1)
    y_test = test[targets_actual]
    test_tag = test[tags]
    
    return (X_train, y_train, X_val, y_val, X_test, y_test,
            train_tag, val_tag, test_tag)


def fit_model(filepath, X_train, y_train, X_val, y_val):
    
    n_rows, n_cols = X_train.shape
    
    model = Sequential()
    model.add(Dense(n_cols, input_dim = n_cols, kernel_initializer='uniform',
                    activation='linear'))
    model.add(Dense(n_cols*5, kernel_initializer='uniform',
                    activation='relu'))
    model.add(Dense(5, kernel_initializer='uniform', activation='linear'))
    
    lrs = [0.001, 0.0001]
    patience_vals = [20, 200]
    epoch_vals = [100, 1000]
    
    for ind in range(len(lrs)):
    
        model.compile(loss='mean_squared_error',
                      optimizer=optimizers.RMSprop(lr=lrs[ind]),
                      metrics=['mean_squared_error'])

        mc = ModelCheckpoint(filepath, monitor='val_mean_squared_error',
                                 verbose=1, save_best_only=True,
                                 mode='min')

        es = EarlyStopping(monitor='val_mean_squared_error',
                           patience=patience_vals[ind], verbose=1,
                           min_delta=0.00001)

        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=epoch_vals[ind], batch_size=1024*256,
                  callbacks=[es, mc],
                  verbose=1)

        model = load_model(filepath)
    
    return model


def evaluate(model, X_test, y_test):
    
    preds = model.predict(X_test)
    a1 = mean_squared_error(preds[:,0].ravel(), y_test['demand'].values)
    a2 = mean_squared_error(preds[:,1].ravel(), y_test['demand+2'].values)
    a3 = mean_squared_error(preds[:,2].ravel(), y_test['demand+3'].values)
    a4 = mean_squared_error(preds[:,3].ravel(), y_test['demand+4'].values)
    a5 = mean_squared_error(preds[:,4].ravel(), y_test['demand+5'].values)

    fitness = np.mean([a1, a2, a3, a4, a5])
    
    return fitness

## GENETIC ALGORITHM FUNCTIONS

In [45]:
def calculate_testmse(data, pctile, window):
    
    train, val, test = create_trainval_test(data=data, pctile=pctile,
                                            max_window= window)
    
    print('Creating Data: Done', end='\t')
    
    (X_train, y_train, X_val, y_val, X_test, y_test,
     train_tag, val_tag, test_tag) = split(train, val, test)
    
    model = fit_model('model.hdf5', X_train, y_train,
                      X_val, y_val)
    
    print('Model Fitting: Done', end='\t')
    
    fitness = evaluate(model, X_test, y_test)
    
    print(f'Fitness = {fitness}')
    
    return fitness

def mate(fittest, mating_pairs, n_children):
    
    species = np.zeros((n_species, len_gene), dtype=int)
    species[0] = fittest

    i = 1   
    for father, mother in mating_pairs: 
        
        parents = [father, mother]
        np.random.shuffle(parents)
        
        child = np.zeros(len_gene, dtype=int)
        
        child[:1] = parents[0][:1]
        child[1:] = parents[1][1:]

        species[i] = child
        i += 1
        if i == n_children: break

    return species

def mutate(species, mutation_rate):
    
    n_species = species.shape[0]
    n_mutate = int(mutation_rate*(n_species - 1))

    inds_mut = np.random.randint(1, n_species, size=n_mutate)
    species[inds_mut, 0] = np.random.choice(np.arange(min_pctile, max_pctile, 5), n_mutate)

    inds_mut = np.random.randint(1, n_species, size=n_mutate)
    species[inds_mut, 1] = np.random.choice(np.arange(min_window, max_window, 4), n_mutate)
   
    return species

def diversify(sp):
    
    col_to_change = np.random.choice(list(range(len_gene)))
    
    if col_to_change == 0:
        sp[0] = np.random.choice(np.arange(min_pctile, max_pctile, 5),1)# PERCENTILE
    
    elif col_to_change == 1:
        sp[1] = np.random.choice(np.arange(min_window, max_window, 4), 1)
    
    return sp

def add_to_records(all_records, species, fitness_values):
    
    res_gen = pd.DataFrame(species, columns=['percentile', 'window'])
    res_gen['gen'] = gen
    res_gen['index'] = list(range(n_species))
    res_gen['fitness_test_mse'] = fitness_values
    
    all_records = pd.concat([all_records, res_gen])
    all_records.to_csv('all_records.csv', index=False)
    
    return all_records

In [17]:
df = pd.read_csv('Traffic Management/training_processed.csv')
df = df.sort_values(['geohash6', 'T_n'])

del df['timestamp'], df['T_n'], df['day']

data = df[['geohash6', 'lat', 'long', 'demand']].copy()
data.head()

,geohash6,lat,long,demand
0,qp02yc,-5.48,90.7,0.020592
1,qp02yc,-5.48,90.7,0.010292
2,qp02yc,-5.48,90.7,0.000000
3,qp02yc,-5.48,90.7,0.000000
4,qp02yc,-5.48,90.7,0.000000


In [12]:
data.shape

(7556911, 4)

In [ ]:
# g = data.groupby('geohash6')
# all_corrs = []
# for pctile in range(10,100, 10):

#     all_gh = []
    
#     for each_gh in g.groups.keys():

#         dummy = g.get_group(each_gh).copy()
#         dummy['demand_fft'] = denoise(dummy.demand.values, pctile)
#         all_gh.append(dummy)
        
#     all_dfs = pd.concat(all_gh)
#     corr = np.corrcoef(all_dfs.demand.values, all_dfs.demand_fft.values)[0,1]
#     all_corrs.append(corr)
    
# fig = plt.figure()
# plt.plot(range(10,100, 10), all_corrs)

In [ ]:
data.shape

## Genetic Algorithm: Implementation

In [18]:
import os
res_file = 'all_records.csv' # Filename of genetic algorithm records
exists = os.path.isfile(res_file)

n_species = 10
len_gene = 2
max_gen = 10
mut_rates = np.linspace(0, 0.5, max_gen)
n_select = 5

min_pctile, max_pctile = 5, 100
min_window, max_window = 4, 100

col_names = ['percentile', 'window', 'gen', 'index', 'fitness_test_mse']
gene_names = ['percentile', 'window']

tested_species = np.array([-99,-99])

The code below initializes the genetic algorithm. When a run is interrupted due to due to unseen circumstances, the user may continue running the GA from most recent generation using the code below.

In [19]:
if exists:
    results_df = pd.read_csv(res_file)
    tested_species = results_df[gene_names].values
    recent_results = results_df[results_df['gen'] == max(results_df['gen'])][col_names].reset_index(drop=True)

    num_generations = list(range(max(results_df['gen'])+1, max_gen))

    mating_pool = recent_results[gene_names].values[:n_select]
    mating_pairs = list(itertools.combinations(mating_pool, 2))    
    np.random.shuffle(mating_pairs)    
    
    recent_results = recent_results.sort_values('fitness_test_mse')
    fittest_val = recent_results.iloc[0]['fitness_test_mse']
    fittest = recent_results.iloc[0][gene_names].values.astype(int)
    
    species = mate(fittest=fittest, mating_pairs=mating_pairs,
               n_children=n_species)
    species = mutate(species, mutation_rate=mut_rates[num_generations[0]])
    
    tested_species = results_df[gene_names].values
    
else:
    
    num_generations = list(range(0,max_gen))
    results_df = pd.DataFrame(columns=col_names)
    
    # INITIALIZE VALUES
    species = np.zeros((n_species, len_gene), dtype=int)
    species[:,0] = np.random.choice(np.arange(min_pctile, max_pctile, 5), n_species)
    species[:,1] = np.random.choice(np.arange(min_window, max_window, 4), n_species)    
    tested_species = np.array([-99,-99])

The code below runs the genetic algorithm from a predefined number of generations. The output below shows a resumed run from the 8th generation (generation index 7) up to the 10th generation.

In [21]:
for gen in num_generations:
    
    print(f'generation {gen}:')
    fitness_values = []
    
    for j, sp in enumerate(species):

        if gen == 0:
            print(sp, end=' ')
            fitness = calculate_testmse(data, sp[0], sp[1])   
            tested_species = np.vstack([tested_species, sp])

        elif gen > 0:
            if j == 0:
                print(sp, 'Done', end='\t')
                fitness = fittest_val
                print(f'Fitness = {fitness}')

            elif j > 0:
                while sp.tolist() in tested_species.tolist():
                    sp = diversify(sp)
                print(sp, end=' ')
                species[j,:] = sp
                fitness = calculate_testmse(data, sp[0], sp[1])    
                tested_species = np.vstack([tested_species, sp])

        fitness_values.append(fitness)

    res_gen = pd.DataFrame(species, columns=gene_names)
    res_gen['gen'] = gen
    res_gen['index'] = list(range(n_species))
    res_gen['fitness_test_mse'] = fitness_values
    
    results_df = pd.concat([results_df, res_gen])
    results_df.to_csv(res_file, index=False)

    fittest, fittest_val = (species[np.argmin(fitness_values)],
                            np.min(fitness_values))
    
    print(f'Fittest:{fittest} {fittest_val}')
    print('==================================')
    
    species = species[np.argsort(fitness_values)]
    mating_pool = species[:n_select]
    mating_pairs = list(itertools.combinations(mating_pool, 2))    
    np.random.shuffle(mating_pairs)
    
    species = mate(fittest=fittest, mating_pairs=mating_pairs,
                   n_children=n_species)
    
    species = mutate(species, mutation_rate=mut_rates[gen])

generation 7:
[45 92] Done	Fitness = 0.0019635655589860857
[30 44] Creating Data: Done	Model Fitting: Done	Fitness = 0.003377516621380486
[75 12] Creating Data: Done	Model Fitting: Done	Fitness = 0.003433179373105824
[85 36] Creating Data: Done	Model Fitting: Done	Fitness = 0.0033305781445787884
[ 5 36] Creating Data: Done	Model Fitting: Done	Fitness = 0.003343668680430529
[95 52] Creating Data: Done	Model Fitting: Done	Fitness = 0.004323591796865457
[75 52] Creating Data: Done	Model Fitting: Done	Fitness = 0.004525261417953867
[45 32] Creating Data: Done	Model Fitting: Done	Fitness = 0.0027725289104957983
[55 80] Creating Data: Done	Model Fitting: Done	Fitness = 0.003492590125429041
[85  8] Creating Data: Done	Model Fitting: Done	Fitness = 0.0024145148705954724
Fittest:[45 92] 0.0019635655589860857
generation 8:
[45 92] Done	Fitness = 0.0019635655589860857
[25  8] Creating Data: Done	Model Fitting: Done	Fitness = 0.0025493469318358325
[60 96] Creating Data: Done	Model Fitting: Done	Fi

## BEST MODEL CONFIGURATIONS

In [26]:
results_all = pd.read_csv('all_records.csv')
top = (results_all.sort_values(['fitness_test_mse', 'window'])
       .drop_duplicates(['percentile', 'window'])).iloc[:5]
top

,percentile,window,gen,index,fitness_test_mse
19,45,92,1,9,0.001964
93,5,96,9,3,0.001974
9,40,92,0,9,0.001994
1,45,96,0,1,0.002042
91,55,92,9,1,0.002108


In [ ]:
best1 = top.iloc[0][['percentile', 'window']].values.astype(int).tolist()

train, val, test = create_trainval_test(data=data, pctile=best1[0],
                                        max_window=best1[1], frac=0.5)

train, val, test = create_trainval_test(data=data, pctile=best1[0],
                                        max_window=best1[1], frac=0.5)

(X_train, y_train, X_val, y_val, X_test, y_test,
 train_tag, val_tag, test_tag) = split(train, val, test)

model = fit_model('model1.hdf5', X_train, y_train,
                  X_val, y_val)

Train on 2228936 samples, validate on 743087 samples
Epoch 1/100
2097152/2228936 [===========================>..] - ETA: 2s - loss: 0.4184 - mean_squared_error: 0.4184
Epoch 00001: val_mean_squared_error improved from inf to 0.02539, saving model to model1.hdf5
2228936/2228936 [==============================] - 54s 24us/step - loss: 0.3952 - mean_squared_error: 0.3952 - val_loss: 0.0254 - val_mean_squared_error: 0.0254
Epoch 2/100
2097152/2228936 [===========================>..] - ETA: 1s - loss: 0.0295 - mean_squared_error: 0.0295
Epoch 00002: val_mean_squared_error did not improve from 0.02539
2228936/2228936 [==============================] - 20s 9us/step - loss: 0.0300 - mean_squared_error: 0.0300 - val_loss: 0.0361 - val_mean_squared_error: 0.0361
Epoch 3/100
2097152/2228936 [===========================>..] - ETA: 1s - loss: 0.0472 - mean_squared_error: 0.0472
Epoch 00003: val_mean_squared_error did not improve from 0.02539
2228936/2228936 [==============================] - 20s 9u

In [ ]:
best2 = top.iloc[1][['percentile', 'window']].values.astype(int).tolist()

train, val, test = create_trainval_test(data=data, pctile=best2[0],
                                        max_window=best2[1], frac=0.5)

train, val, test = create_trainval_test(data=data, pctile=best2[0],
                                        max_window=best2[1], frac=0.5)

(X_train, y_train, X_val, y_val, X_test, y_test,
 train_tag, val_tag, test_tag) = split(train, val, test)

model = fit_model('model2.hdf5', X_train, y_train,
                  X_val, y_val, X_test, y_test)

## BASELINE ERROR

In [40]:
from sklearn.metrics import mean_squared_error

# BASELINE MSE OF T + 1
b1 = mean_squared_error(test['demand'].values,
                        test['demand-1'].values)

# BASELINE MSE OF T + 2
b2 = mean_squared_error(test['demand'].values,
                        test['demand-2'].values)

# BASELINE MSE OF T + 3
b3 = mean_squared_error(test['demand'].values,
                        test['demand-3'].values)

# BASELINE MSE OF T + 4
b4 = mean_squared_error(test['demand'].values,
                        test['demand-4'].values)

# BASELINE MSE OF T + 5
b5 = mean_squared_error(test['demand'].values,
                        test['demand-5'].values)

b1, b2, b3, b4, b5

(0.0275126235967825,
 0.034559915748419066,
 0.04001677974234045,
 0.044955243093510726,
 0.04993596338276819)

In [27]:
np.mean([b1, b2, b3, b4, b5])

0.039370885265768776

## REMOVE VERBOSITY OF NEURAL NETWORK